In [1]:
from fractions import Fraction

In [2]:
f = Fraction(2, 3)
f.denominator

3

In [3]:
f.numerator

2

In [4]:
f.speak() #shouldnt work

AttributeError: 'Fraction' object has no attribute 'speak'

In [11]:
Fraction.speak = lambda self, msg : f"hello {msg}"

In [13]:
f.speak("Thomas")

'hello Thomas'

In [14]:
Fraction.is_integral = lambda self: self.denominator == 1

In [21]:
f1 = Fraction(2, 3)
f2 = Fraction(64, 8)

In [22]:
f1.is_integral()

False

In [23]:
f2.is_integral()

True

In [24]:
f1

Fraction(2, 3)

In [25]:
f2

Fraction(8, 1)

In [27]:
def dec_speak(cls):
    cls.speak = lambda self, msg: f"{self.__class__.__name__} says {msg}"
    return cls

In [28]:
Fraction = dec_speak(Fraction)

In [29]:
f1 = Fraction(2, 3)

In [31]:
f1.speak("hello")

'Fraction says hello'

In [32]:
class Person:
    pass

In [33]:
Person = dec_speak(Person)

In [34]:
p = Person()

In [35]:
p.speak("This works!!!")

'Person says This works!!!'

In [36]:
from datetime import datetime, timezone

In [40]:
def info(obj):
        results = []
        results.append(f"time: {datetime.now(timezone.utc)}")
        results.append(f"class: {obj.__class__.__name__}")
        results.append(f"id: {hex(id(obj))}")
        for k, v in vars(obj).items():
            results.append(f"{k}: {v}")
        return results


def debug_info(cls): # warning no closure going on...    
    cls.debug = info
    return cls

In [43]:
@debug_info
class Person:
    def __init__(self, name, birth_year):
        self.name = name
        self.birth_year = birth_year
    
    def say_hi():
        return "Hello there!"

In [45]:
p = Person("Thomas", 1991)
p.debug()

['time: 2023-04-12 08:00:21.861081+00:00',
 'class: Person',
 'id: 0x106f3a890',
 'name: Thomas',
 'birth_year: 1991']

In [57]:
# without returning the class

def info(obj):
        results = []
        results.append(f"time: {datetime.now(timezone.utc)}")
        results.append(f"class: {obj.__class__.__name__}")
        results.append(f"id: {hex(id(obj))}")
        for k, v in vars(obj).items():
            results.append(f"{k}: {v}")
        return results


def debug_info(cls): # warning no closure going on...    
    cls.debug = info
    return cls

class Person:
    def __init__(self, name, birth_year):
        self.name = name
        self.birth_year = birth_year
    
    def say_hi():
        return "Hello there!"

debug_info(Person)

__main__.Person

In [58]:
p = Person("Thomas", 1991)

In [59]:
p.debug()

['time: 2023-04-12 08:11:42.315497+00:00',
 'class: Person',
 'id: 0x1073ee250',
 'name: Thomas',
 'birth_year: 1991']

In [60]:
@debug_info
class Automobile:
    def __init__(self, make, model, year, top_speed):
        self.make = make
        self.model = model
        self.year = year
        self.top_speed = top_speed
        self._speed = 0
    
    @property # this is a decorator
    def speed(self):
        return self._speed
    
    @speed.setter
    def speed(self, new_speed):
        if new_speed > self.top_speed:
            raise ValueError("Cannot exceed top speed")
        else:
            self._speed = new_speed

In [61]:
favorite = Automobile("Ford", "Model T", 1908, 45)

In [62]:
favorite.debug()

['time: 2023-04-12 08:14:42.516530+00:00',
 'class: Automobile',
 'id: 0x1074b4e50',
 'make: Ford',
 'model: Model T',
 'year: 1908',
 'top_speed: 45',
 '_speed: 0']

In [63]:
favorite.speed = 100

ValueError: Cannot exceed top speed

In [64]:
from math import sqrt

In [66]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __abs__(self):
        return sqrt(self.x ** 2 + self.y ** 2)
    
    def __repr__(self) -> str:
        return f"Point({self.x}, {self.y})"

In [67]:
p1, p2, p3  = Point(2, 3), Point(2, 3), Point(0,0)

In [68]:
abs(p1)

3.605551275463989

In [69]:
p1 is p2

False

In [70]:
p2 is p3

False

In [71]:
p1 == p2

False

In [82]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __abs__(self):
        return sqrt(self.x ** 2 + self.y ** 2)
    
    def __eq__(self, other):
        if isinstance(other, Point):
            return self.x == other.x and self.y == other.y
        return False
    
    def __lt__(self, other):
        if isinstance(other, Point):
            return abs(self) < abs(other)
        return NotImplemented
    
    def __le__(self, other):
        pass
    
    def __gt__(self, other):
        pass
    
    def __ge__(self, other):
        pass
    
    

In [76]:
p1, p2, p3  = Point(2, 3), Point(2, 3), Point(0,0)

In [77]:
p1 == p2

True

In [78]:
p3 < p1

True

In [79]:
p4 = Point(100, 100)

In [80]:
p4 < p1

False

In [81]:
p4 > p1

True

In [83]:
# We are going to assume lt/eq are defined
# We are going to create a decorator that monkeypatches the others

# a <= b if a < b or a == b
# a < b if not(a<b) and a != b
# a >=b if not(a<b)

In [86]:
def complete_ordering(cls):
    if "__eq__" in dir(cls) and "__lt__" in dir(cls):
        cls.__le__ = lambda self, other: self < other or self == other
        cls.__gt__ = lambda self, other: not (self < other) and not (self == other)
        cls.__ge__ = lambda self, other: not(self < other)
    return cls

In [87]:
@complete_ordering
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __abs__(self):
        return sqrt(self.x ** 2 + self.y ** 2)
    
    def __eq__(self, other):
        if isinstance(other, Point):
            return self.x == other.x and self.y == other.y
        return False
    
    def __lt__(self, other):
        if isinstance(other, Point):
            return abs(self) < abs(other)
        return NotImplemented

In [88]:
p1, p2, p3, p4  = Point(2, 3), Point(2, 3), Point(0,0), Point(100, 100)

In [89]:
p1 <= p4

True

In [90]:
p1 != p2

False

In [91]:
from functools import total_ordering

@total_ordering
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __abs__(self):
        return sqrt(self.x ** 2 + self.y ** 2)
    
    def __eq__(self, other):
        if isinstance(other, Point):
            return self.x == other.x and self.y == other.y
        return False
    
    def __lt__(self, other):
        if isinstance(other, Point):
            return abs(self) < abs(other)
        return NotImplemented

In [92]:
p1, p2, p3, p4  = Point(2, 3), Point(2, 3), Point(0,0), Point(100, 100)

In [93]:
p1 <= p2

True

In [94]:
p1 >= p4

False